In [8]:
import spacy
from spacy.lang.es import Spanish
import medspacy
from glob import glob
from spacy.tokens import Span
from medspacy.visualization import visualize_dep, visualize_ent
from medspacy.context import ConTextRule, ConTextComponent
from medspacy.target_matcher import TargetMatcher, TargetRule

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

D:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
nlp = spacy.load("es_core_news_md", disable=["ner"]) #loading a Spanish model

In [4]:
target_matcher = TargetMatcher(nlp)
Span.set_extension("CIE10", default="", force=True)
rules = TargetRule.from_json("./data/cie10_rules.jsonl") # file with Spanish rules
target_matcher.add(rules)
nlp.add_pipe(target_matcher)

In [ ]:
context_rules = [ 

    ConTextRule("no", "NEGATED_EXISTENCE", direction="FORWARD" ) 

] 

In [5]:
context = ConTextComponent(nlp, rules=None)
#context = ConTextComponent(nlp, rules="default")
context_rules = ConTextRule.from_json("./data/context_rules.jsonl")
#print (context_rules)
context.add(context_rules)
nlp.add_pipe(context)

In [6]:
nlp.pipe_names

['tagger', 'parser', 'target_matcher', 'context']

In [9]:
inf_nuevos = glob("./data/informes/*.txt")

with open("./data/informes_nuevos.txt", "w", encoding="utf-8") as ant_per_inf:
    for ap_infor in inf_nuevos:
        with open(ap_infor, "r", encoding="utf-8") as ap:
            ants_per = ap.read().split("\n\n")[1:2]
            for ant_per in ants_per:
                ant_per_inf.write(ant_per)

informe_urgencias = []
with open ("./data/informes_nuevos.txt", "r", encoding="utf-8") as f:
    informes_urgencias = f.read().split("\n\n")
    for informe in informes_urgencias:
        informe = informe.replace("-", "")
        informe = informe.replace("*", "")
        #informe = informe.replace("\n", "")
        #informe = informe.lower()
        #informe = informe.replace("antecedentes personales:", "")
        #informe = informe.replace("ap:", "")
        #informe = informe.replace("a. personales:", "")
        informe = informe.strip()
        informe_urgencias.append(informe)
informes_urgencias = str(informe_urgencias)    
print (informe_urgencias)  

['ANTECEDENTES PERSONALES:\n No RAMc\n No FRCV\n Exfumadora desde hace 17 años de algo más de medio paq/día. No hábitos tóxicos.\n No enfermedades de interés.\n IQx: Apendicectomía\n Tto habitual (actualizado a 10/12): BELARA 2 MG/0,03 MG 100, ESPIDIFEN 600 MG 111 a demanda, AERIUS 5MG 100, actualmente Paracetamol/Codeína.AP:\nNRAMC. \n Fumadora de 20 cig/día durante 15 años. No hábito enólico. \n No HTA. No DM. No DL. \nSeguimiento en Oncología (xxxx) por Carcinoma de mama metacrónico: \nxxxx: Carcinoma in situ de mama derecha tratado con Tumorectomía + Radioterapia. 2015: CARCINOMA DUCTAL INFILTRANTE, grado I, pT1c, V0, L0, N1mi, (sn, método OSNA), RRHH positivos, cerb2 neg. Ki 67: 15% en paciente premenopausica tratada con Mastectomía con RI + BSGC (6.11.2015). \nONCOTYPE Dx bajo riesgo.\nNO PORTADORA DE MUTACIÓN DELETEREA EN NINGUNO DE LOS GENES DE SUSCEPTIBILIDAD AL CANCER DE OVARIO Y MAMA\nA. PERSONALES:\nNO alergias medicamentosas conocidas. \nAsma extrínseca intermitente. Con t

In [ ]:
text = "No HTA desde hace 5 años. No DM y si asma. Cirrosis grave. Oxigeno domiciliario previo"

In [12]:
doc = nlp(informes_urgencias)
ante_encontrados = [[ent.text, ent._.CIE10] for ent in doc.ents if ent._.is_negated == False]
print (f"Número de antecedentes encontrados: {len(ante_encontrados)}\n")
print (ante_encontrados) 

Número de antecedentes encontrados: 10

[['Fumadora', 'F17.210'], ['Carcinoma', 'C07'], ['Carcinoma', 'C07'], ['CARCINOMA DUCTAL INFILTRANTE', 'C50.9'], ['HTA', 'I10'], ['DM', 'E11.9'], ['HTA', 'I10'], ['DM tipo 2', 'E11.9'], ['Cardiopatía isquémica', 'I25.9'], ['Exfumadora', 'Z87.891']]


In [ ]:
tipe

In [ ]:
doc = nlp(text)
for ent in doc.ents:
    if ent._.is_negated == False:
        ante_encontrados2 = [ent.text, ent._.CIE10]
ante_encontrados2

In addition to linking targets and modifiers, ConText will also set attributes for each entity:

In [ ]:
doc = nlp(text)
for ent in doc.ents:
    if any([ent._.is_negated, ent._.is_uncertain, ent._.is_historical, ent._.is_family, ent._.is_hypothetical, ]):
        print("'{0}' modified by {1} in: '{2}'".format(ent, ent._.modifiers, ent.sent))